In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import geojsonio
from shapely.geometry import Point, Polygon

In [2]:
buildings = gpd.read_file("data/uk_osm_building_extract.geojson")
parcels = gpd.read_file("data/uk_land_registry_parcel_extract.geojson")

In [3]:
def ParcelAssign(buildingdata,propertydata):
    
    #Convert from lat-lon to x-y
    buildingdata = buildingdata.to_crs(epsg=27700)
    propertydata = propertydata.to_crs(epsg=27700) 
    
    #Optional: update dataframe with each loop
    buildingdata['Overlapped Parcels'] = ""
    buildingdata['Overlapped Fraction'] = ""
    
    #For each building, iterate over all properties                                  
    for b_index in range(len(buildingdata)): 
        
        #reset for every building                                
        count = 0
        parcel_list = []
        frac_list = []
        
        for p_index in range(len(propertydata)):
        
            if(buildingdata['geometry'][b_index].overlaps(propertydata['geometry'][p_index]))== True:
                
                #calculate overlapping area
                intersection = buildingdata['geometry'][b_index].intersection(propertydata['geometry'][p_index])
                overlap_area = intersection.area
                
                #overlap_area = gpd.overlay(buildingdata['geometry'][b_index],propertydata['geometry'][p_index], how="intersection")
                frac_overlap = overlap_area / buildingdata['geometry'][b_index].area                   
                count+=1
                
                #To allow for cases with multiple parcels
                parcel_list.append(p_index)
                frac_list.append(frac_overlap)
            
        parcel_series = pd.Series(parcel_list)
        frac_series = pd.Series(frac_list)

        buildingdata['Overlapped Parcels'][b_index] = parcel_series
        buildingdata['Overlapped Fraction'][b_index] = frac_series

    return buildingdata

In [4]:
buildings

,fid,full_id,osm_id,osm_type,alt_name,building,building:levels,heritage,heritage:operator,listed_status,...,kiosk,beds,telephone,highway,addr:unit,indoor,room,source:maxspeed,swimming,geometry
0,10,r1132065,1132065,relation,,residential,10,,,,...,,,,,,,,,,"POLYGON ((-1.89807 52.47270, -1.89816 52.47278..."
1,12,r1132398,1132398,relation,,yes,,,,,...,,,,,,,,,,"POLYGON ((-1.89651 52.47320, -1.89609 52.47338..."
2,32,r1222758,1222758,relation,,residential,,,,,...,,,,,,,,,,"POLYGON ((-1.89803 52.46587, -1.89742 52.46600..."
3,45,r3227190,3227190,relation,,residential;student_accommodation,7,,,,...,,,,,,,,,,"POLYGON ((-1.90052 52.46682, -1.90037 52.46679..."
4,538,w73038255,73038255,way,,church,,6,Birmingham City Council,local_grade_B,...,,,,,,,,,,"POLYGON ((-1.90030 52.47232, -1.90034 52.47231..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,7859,w834171777,834171777,way,,residential,,,,,...,,,,,,,,,,"POLYGON ((-1.89628 52.46819, -1.89616 52.46816..."
257,7864,w846463281,846463281,way,,construction,,,,,...,,,,,,,,,,"POLYGON ((-1.89484 52.47345, -1.89460 52.47337..."
258,7936,w908434023,908434023,way,,yes,,,,,...,,,,,,,,,,"POLYGON ((-1.89330 52.47217, -1.89307 52.47210..."
259,7937,w908434026,908434026,way,,yes,,,,,...,,,,,,,,,,"POLYGON ((-1.89350 52.47192, -1.89342 52.47200..."


In [5]:
newbuildings = ParcelAssign(buildings,parcels)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

In [6]:
newbuildings

,fid,full_id,osm_id,osm_type,alt_name,building,building:levels,heritage,heritage:operator,listed_status,...,telephone,highway,addr:unit,indoor,room,source:maxspeed,swimming,geometry,Overlapped Parcels,Overlapped Fraction
0,10,r1132065,1132065,relation,,residential,10,,,,...,,,,,,,,"POLYGON ((407020.782 286090.542, 407014.916 28...","Series([], dtype: float64)","Series([], dtype: float64)"
1,12,r1132398,1132398,relation,,yes,,,,,...,,,,,,,,"POLYGON ((407126.772 286145.376, 407155.483 28...","Series([], dtype: float64)","Series([], dtype: float64)"
2,32,r1222758,1222758,relation,,residential,,,,,...,,,,,,,,"POLYGON ((407024.848 285330.767, 407065.822 28...","Series([], dtype: float64)","Series([], dtype: float64)"
3,45,r3227190,3227190,relation,,residential;student_accommodation,7,,,,...,,,,,,,,"POLYGON ((406854.989 285435.537, 406865.325 28...","Series([], dtype: float64)","Series([], dtype: float64)"
4,538,w73038255,73038255,way,,church,,6,Birmingham City Council,local_grade_B,...,,,,,,,,"POLYGON ((406869.656 286047.762, 406866.614 28...","Series([], dtype: float64)","Series([], dtype: float64)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,7859,w834171777,834171777,way,,residential,,,,,...,,,,,,,,"POLYGON ((407142.900 285588.766, 407151.133 28...",0 58 1 62 dtype: int64,0 0.083388 1 0.191076 dtype: float64
257,7864,w846463281,846463281,way,,construction,,,,,...,,,,,,,,"POLYGON ((407240.033 286173.659, 407256.260 28...","Series([], dtype: float64)","Series([], dtype: float64)"
258,7936,w908434023,908434023,way,,yes,,,,,...,,,,,,,,"POLYGON ((407345.127 286031.098, 407360.409 28...","Series([], dtype: float64)","Series([], dtype: float64)"
259,7937,w908434026,908434026,way,,yes,,,,,...,,,,,,,,"POLYGON ((407331.664 286003.514, 407336.622 28...",0 25 dtype: int64,0 0.041116 dtype: float64
